# 2023_08_08


### binary cross entropy

현재 예측값이 pretrained 모델의 레이블 정보를 가지고 있어서 여러 개의 label이 나타남  
이를 loss function을 binary cross entropy로 변경해서 학습 진행  

> https://discuss.huggingface.co/t/binary-semantic-segmentation-using-segformer/25745/7

### Feature Extractor

현재 SegformerFeatureExtractor를 사용하는 이게 실제 연어 데이터에 대해서 색상을 부각시키고 배경을 없앰  
이를 확인해서 정상적으로 학습되도록 만듬  

### image size
현재 image 사이즈가 작아서 학습이 제대로 안되는 부분이 있음  
이를 SegformerFeatureExtractor를 현재 256 * 256 크기의 이미지를 512 * 512 혹은 1024 * 1024 이미지에서 학습 진행  

### change iou
현재 iou를 foreground(fish)에 대해서만 계산을 함  
이를 background와 foregournd에 대해서도 계산을 하도록 변경하고  
학습시에도 tensorboard에 background와 foregroud에 대해서 기록 되도록 변경   

### binary cross entropy

기본적으로 loss는 BCEWITHLOGITSLOSS를 사용함  
loss를 바꾸는 코드가 없어서 classifier를 binary로 바꿈  

> loss : https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html

> https://discuss.huggingface.co/t/binary-semantic-segmentation-using-segformer/25745/7


model config를 pretrained model config를 사용하고 있었음  

> result : http://202.31.200.123:8888/notebooks/NPLAB-NAS/Members/SEO/salom_ai/infernce.ipynb#2023_08_04

### b5 model

> https://huggingface.co/docs/transformers/main/model_doc/segformer

hugging face의 공식 홈페이지에서는 다음과 같은 pre-trained 모델을 사용이 가능함  

| Model variant | Depths        | Hidden sizes        | Decoder hidden size | Params (M) | ImageNet-1k Top 1 |
|---------------|---------------|---------------------|---------------------|------------|-------------------|
| MiT-b0        | [2, 2, 2, 2]  | [32, 64, 160, 256]  | 256                 | 3.7        | 70.5              |
| MiT-b1        | [2, 2, 2, 2]  | [64, 128, 320, 512] | 256                 | 14.0       | 78.7              |
| MiT-b2        | [3, 4, 6, 3]  | [64, 128, 320, 512] | 768                 | 25.4       | 81.6              |
| MiT-b3        | [3, 4, 18, 3] | [64, 128, 320, 512] | 768                 | 45.2       | 83.1              |
| MiT-b4        | [3, 8, 27, 3] | [64, 128, 320, 512] | 768                 | 62.6       | 83.6              |
| MiT-b5        | [3, 6, 40, 3] | [64, 128, 320, 512] | 768                 | 82.0       | 83.8              |

실제로 사용을 해봤을 때 MiT-b5 모델은 사용이 불가능 했음  


hugging face의 Model에서 `nvidia/segformer-b5-finetuned-cityscapes-1024-1024` 다음 모델을 사용함   

> https://huggingface.co/models?search=segformer_b5

다른 모델을 사용이 가능하나 우선은 확인 용도로 위 모델을 사용  

### Feature Extractor

> https://huggingface.co/docs/transformers/main/model_doc/segformer#transformers.SegformerImageProcessor

| 옵션명 | 설명 |
|--|--|
| do_resize | 이미지의 (높이, 너비) 차원을 지정된 크기로 조정할지 여부 |
| size | 크기 조정 후 출력 이미지의 크기 |
| resample | 이미지 크기를 조정할 때 사용할 리샘플링 필터 |
| do_rescale | 지정된 스케일로 이미지를 재조정할지 여부 |
| rescale_factor | 이미지를 정규화할지 여부 |
| do_normalize | 이미지를 정규화할지 여부 |
| image_mean | 이미지를 정규화할 때 사용할 평균 |
| image_std | 이미지를 정규화할 때 사용할 표준 편차 |
| do_reduce_labels | 세그멘테이션 맵의 모든 레이블 값을 1만큼 줄일지 여부 |


다음과 같은 옵션을 선택을 해서 셋팅이 가능함  

> http://202.31.200.123:8888/notebooks/NPLAB-NAS/Members/SEO/salom_ai/dataloader_check.ipynb#2023_08_08


### change iou

test loop가 끝날때 각 category 별로 iou 를 metric으로 넘김  
tensorborad에도 각 background와 foreground에 대해서도 log를 남기도록 수정  


``` python
class SegformerFinetuner(pl.LightningModule):    
    def __init__(self, id2label, pre_trained ,  train_dataloader=None, val_dataloader=None, test_dataloader=None, metrics_interval=100):
        super(SegformerFinetuner, self).__init__()
        print(f"pre_trained_mode : {pre_trained}")
        self.id2label = id2label
        self.metrics_interval = metrics_interval
        self.train_dl = train_dataloader
        self.val_dl = val_dataloader
        self.test_dl = test_dataloader
        
        self.num_classes = len(id2label.keys())
        self.label2id = {v:k for k,v in self.id2label.items()}
        
        
        print(self.id2label)
        print(self.label2id)
        
        self.model = SegformerForSemanticSegmentation.from_pretrained(
            pre_trained, 
            return_dict=False, 
            num_labels=self.num_classes,
            id2label=self.id2label,
            label2id=self.label2id,
            ignore_mismatched_sizes=True,
        )
        
        
        self.train_mean_iou = load_metric("mean_iou")
        self.val_mean_iou = load_metric("mean_iou")
        self.test_mean_iou = load_metric("mean_iou")
                
    def forward(self, images, masks):
        outputs = self.model(pixel_values=images, labels=masks)
        return(outputs)
    
    def training_step(self, batch, batch_nb):
        images, masks = batch['pixel_values'], batch['labels']
        outputs = self(images, masks)
        loss, logits = outputs[0], outputs[1]
        upsampled_logits = nn.functional.interpolate(
            logits, 
            size=masks.shape[-2:], 
            mode="bilinear", 
            align_corners=False
        )

        predicted = upsampled_logits.argmax(dim=1)
        self.train_mean_iou.add_batch(
            predictions=predicted.detach().cpu().numpy(), 
            references=masks.detach().cpu().numpy()
        )
        
        if batch_nb % self.metrics_interval == 0:

            metrics = self.train_mean_iou.compute(
                num_labels=self.num_classes, 
                ignore_index=255, 
                reduce_labels=False,
            )
            
            metrics = {'loss': loss, "mean_iou": metrics["mean_iou"], "mean_accuracy": metrics["mean_accuracy"]}
            
            
            for k,v in metrics.items():
                self.log(k,v)
            
            return(metrics)
        else:
            return({'loss': loss})
    
    def validation_step(self, batch, batch_nb):
        images, masks = batch['pixel_values'], batch['labels']
        outputs = self(images, masks)
        loss, logits = outputs[0], outputs[1]
        upsampled_logits = nn.functional.interpolate(
            logits, 
            size=masks.shape[-2:], 
            mode="bilinear", 
            align_corners=False
        )
        predicted = upsampled_logits.argmax(dim=1)
        self.val_mean_iou.add_batch(
            predictions=predicted.detach().cpu().numpy(), 
            references=masks.detach().cpu().numpy()
        )
        self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

    def on_validation_epoch_end(self):
        metrics = self.val_mean_iou.compute(
            num_labels=self.num_classes, 
            ignore_index=255, 
            reduce_labels=False,
        )
        
        ### add code ###
        val_mean_iou = metrics["mean_iou"]
        val_mean_accuracy = metrics["mean_accuracy"]
        val_background_iou = metrics['per_category_iou'][0]
        val_foreground_iou = metrics['per_category_iou'][1]
        
        self.log("val_mean_iou", val_mean_iou, prog_bar=True, logger=True)
        self.log("val_background_iou", val_background_iou, prog_bar=True, logger=True)
        self.log("val_foreground_iou", val_foreground_iou, prog_bar=True, logger=True)
        self.log("val_mean_accuracy", val_mean_accuracy, prog_bar=True, logger=True)
        ################
    
    def test_step(self, batch, batch_nb):
        
        images, masks = batch['pixel_values'], batch['labels']
        outputs = self(images, masks)
        loss, logits = outputs[0], outputs[1]
        upsampled_logits = nn.functional.interpolate(
            logits, 
            size=masks.shape[-2:], 
            mode="bilinear", 
            align_corners=False
        )
        predicted = upsampled_logits.argmax(dim=1)
        self.test_mean_iou.add_batch(
            predictions=predicted.detach().cpu().numpy(), 
            references=masks.detach().cpu().numpy()
        )
        return({'test_loss': loss})
    
    def test_epoch_end(self, outputs):
        metrics = self.test_mean_iou.compute(
              num_labels=self.num_classes, 
              ignore_index=255, 
              reduce_labels=False,
          )
        avg_test_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        test_mean_iou = metrics["mean_iou"]
        test_mean_accuracy = metrics["mean_accuracy"]
        
        ### add code ###
        metrics = {"test_loss": avg_test_loss, 
                   "test_mean_iou":test_mean_iou, 
                   "test_mean_accuracy":test_mean_accuracy,
                   "backgournd_iou" : metrics['per_category_iou'][0] , 
                   "foregournd_iou" : metrics['per_category_iou'][1]
                   }
        ################
        for k,v in metrics.items():
            self.log(k,v)
        
        return metrics
    
    def configure_optimizers(self):
        return torch.optim.Adam([p for p in self.parameters() if p.requires_grad], lr=2e-05, eps=1e-08)
    
    def train_dataloader(self):
        return self.train_dl
    
    def val_dataloader(self):
        return self.val_dl
    
    def test_dataloader(self):
        return self.test_dl
```

### change iou

miou 계산 방식을 hugging face의 `load_metric('mean_iou')`로 변경해서 확인

``` python 
results = load_metric('mean_iou').compute(predictions=[predicted[i]], 
                                      references=[batch['labels'][i]], 
                                      num_labels=2, 
                                      ignore_index=255)
```

> infernce notebook : http://202.31.200.123:8888/notebooks/NPLAB-NAS/Members/SEO/salom_ai/infernce.ipynb#2023_08_08

### normalize off

normalize의 옵션을 끄고 학습을 하고 추론

> tensorboard : http://202.31.200.70:8888/notebooks/NPLAB-NAS/Members/SEO/salom_ai/tensorboard.ipynb

> infercne : http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/salom_ai/infernce.ipynb#2023_08_09

### Data augmentation

학습용 데이터가 310장 정도여서 데이터가 부족한것 같아서 다음과 같은 데이터 증강을 이용해서 데이터를 더 확보를 진행  

> transform test : http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/salom_ai/dataloader_check.ipynb#transform-check

### DeepFish network

* haven-ai install : pip install --upgrade git+https://github.com/haven-ai/haven-ai

> dataloader visualization : http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/salom_ai/DeepFish/scripts/training.ipynb#visualize-dataset

> training : http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/salom_ai/DeepFish/scripts/training.ipynb#training

> infrecne : http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/salom_ai/DeepFish/scripts/training.ipynb#model-load-&-infrernce